### Accessing aggregate statistics of the ESDC before, during and after Natural desasters

In this notebook we demonstrate how users can easily access time series before and after an event that was reported in
the [EM-DAT database](https://www.emdat.be/). The datacube data was aggregated for several variables over the municipalities where the event was reported. 
Time series were generated starting from 64 days prior to the reported start of the event until 64 days after the reported start of the event. An additional 
aggregation was done for different land cover types to appreciate the fact that different ecosystems like grasslands, forests and agricultural areas respond differently to events like droughts and heatwaves. 

As a note of caution: The data here are strictly aggregated to the administrative units where the event happened. This does not guarantee, that the actual event will be captured since there might be a mismatch between the actual event extent and the admin unit where the event occured. .

### Loading the aggregated data

First we access the aggregated data from the publicly available S3 Store and load them into an xarray

In [ ]:
import s3fs
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

s3 = s3fs.S3FileSystem(anon=True,client_kwargs = {'endpoint_url' : "https://obs.eu-de.otc.t-systems.com/"})
st = s3fs.S3Map(root="obs-esdc-v2.1/esdc-aggregated-emdat.zarr", s3=s3, check=False)
ds = xr.open_zarr(st)

This returns an xarray dataset each variables having the followinig dimensions:
    
- **Event** This denotes the different events contained in the EM-DAT database
- **Landcover** Landcover type over which the aggregation was performed
- **Time** Time in days relative to the start of the event
- **Units** denotes if the aggregation was done on the raw data or on z-transformed anomalies

In [ ]:
ds

Assume now we are interested in the drought in China from 2006. From the EM-DAT Database we get the year (2006), sequence number (9741) and ISO code (CHN) of the country. The we can subset the dataset to this event:

In [ ]:
chn_2006 = ds.sel(Event=b'2006_9741_CHN')

Since this was a drought, let us look at the time series for the surface moisture prior to the event and after the start of the event for different land cover tyoes:

In [ ]:
chn_2006.sel(Units=b'Original').surface_moisture.plot.line(x='Time')
lcnames = [v.decode("utf8") for v in chn_2006.Landcover.values]
plt.legend(lcnames,loc='upper left')
plt.title("Surface moisture in different land cover types before and after start of the reported drought");

One can see that for aggregated over all land cover types the surface moisture seems to be quite stable. However, splitting the data by landcover shows the reduced moisture in grasslands which reverted to higher values a few weeks after the start of the event. 